In [1]:
import requests
import json

# Define admin API URLs for Alice and Bob
CENTRAL_PARTY_ADMIN_URL = "http://localhost:8001"
FL_CLIENT_ADMIN_URL = "http://localhost:8101"


def create_invitation(admin_url):
    """Create a new invitation."""
    response = requests.post(f"{admin_url}/connections/create-invitation")
    return response.json()


def receive_invitation(admin_url, invitation):
    """Receive an invitation."""
    response = requests.post(
        f"{admin_url}/connections/receive-invitation", json=invitation
    )
    return response.json()


def send_message(admin_url, connection_id, message):
    """Send a message to a connection."""
    response = requests.post(
        f"{admin_url}/connections/{connection_id}/send-message",
        json={"content": message},
    )
    return response.json()


def get_messages(admin_url, connection_id):
    """Get all messages for a connection."""
    response = requests.get(f"{admin_url}/connections/{connection_id}/messages")
    return response.json()


def get_connection(admin_url, connection_id):
    """Get the connection details."""
    response = requests.get(f"{admin_url}/connections/{connection_id}")
    return response.json()

In [2]:
# Step 1: Create an invitation from Central Party
print("Central Party: Creating invitation...")
central_party_invitation = create_invitation(CENTRAL_PARTY_ADMIN_URL)
print("Central Party's invitation:", json.dumps(central_party_invitation, indent=2))

Central Party: Creating invitation...
Central Party's invitation: {
  "connection_id": "ef42804a-5bb0-45b6-9d04-e9d3f2e76cff",
  "invitation": {
    "@type": "did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/invitation",
    "@id": "caec5ea4-d8d7-40ac-850a-7c6ba9466735",
    "label": "Central Party Agent",
    "serviceEndpoint": "http://central-party-agent:8000",
    "recipientKeys": [
      "BcUaLSKKsVsYCsd4CNabu4aTvp5776ZJYpg4DG1ZnHE"
    ]
  },
  "invitation_url": "http://central-party-agent:8000?c_i=eyJAdHlwZSI6ICJkaWQ6c292OkJ6Q2JzTlloTXJqSGlxWkRUVUFTSGc7c3BlYy9jb25uZWN0aW9ucy8xLjAvaW52aXRhdGlvbiIsICJAaWQiOiAiY2FlYzVlYTQtZDhkNy00MGFjLTg1MGEtN2M2YmE5NDY2NzM1IiwgImxhYmVsIjogIkNlbnRyYWwgUGFydHkgQWdlbnQiLCAic2VydmljZUVuZHBvaW50IjogImh0dHA6Ly9jZW50cmFsLXBhcnR5LWFnZW50OjgwMDAiLCAicmVjaXBpZW50S2V5cyI6IFsiQmNVYUxTS0tzVnNZQ3NkNENOYWJ1NGFUdnA1Nzc2WkpZcGc0REcxWm5IRSJdfQ=="
}


In [3]:
# Step 2: FL Client receives Central Party's invitation
print("FL Client: Receiving invitation...")
fl_client_connection = receive_invitation(
    FL_CLIENT_ADMIN_URL, central_party_invitation["invitation"]
)
print("FL Client's connection:", json.dumps(fl_client_connection, indent=2))

FL Client: Receiving invitation...
FL Client's connection: {
  "my_did": "ERPBrsjzB17jgnoAWVJzJR",
  "connection_id": "d56aa5e5-f9c8-42b4-a078-d7deffeca316",
  "invitation_msg_id": "caec5ea4-d8d7-40ac-850a-7c6ba9466735",
  "invitation_mode": "once",
  "updated_at": "2024-07-05 04:28:36.440894Z",
  "their_label": "Central Party Agent",
  "rfc23_state": "request-sent",
  "request_id": "86f029e6-95bf-4766-a405-00a621ae44de",
  "their_role": "inviter",
  "routing_state": "none",
  "invitation_key": "BcUaLSKKsVsYCsd4CNabu4aTvp5776ZJYpg4DG1ZnHE",
  "accept": "auto",
  "state": "request",
  "connection_protocol": "connections/1.0",
  "created_at": "2024-07-05 04:28:36.438508Z"
}


In [4]:
# Check the connection status
fl_client_connection_id = fl_client_connection["connection_id"]
fl_client_connection = get_connection(FL_CLIENT_ADMIN_URL, fl_client_connection_id)
print(
    "FL Client's connection status:",
    json.dumps(fl_client_connection["state"], indent=2),
)

central_party_connection_id = central_party_invitation["connection_id"]
central_party_connection = get_connection(
    CENTRAL_PARTY_ADMIN_URL, central_party_connection_id
)
print(
    "Central Party's connection status:",
    json.dumps(central_party_connection["state"], indent=2),
)

FL Client's connection status: "request"
Central Party's connection status: "response"


In [5]:
# Step 3: FL Client sends a message to Central Party
print("FL Client: Sending message to Central Party...")
message_response = send_message(
    FL_CLIENT_ADMIN_URL, fl_client_connection_id, "Hello Alice, this is Bob!"
)
print("Message response:", json.dumps(message_response, indent=2))

FL Client: Sending message to Central Party...
Message response: {}


In [6]:
# Step 4: Central Party sends a message to FL Client
print("FL Client: Sending message to Central Party...")
message_response = send_message(
    CENTRAL_PARTY_ADMIN_URL, central_party_connection_id, "Test message"
)
print("Message response:", json.dumps(message_response, indent=2))

FL Client: Sending message to Central Party...
Message response: {}
